# Example to use HW-NAS-Bench under NAS-Bench-201's Space

In [ ]:
import matplotlib.pyplot as plt

def plot_histogram(HW_metrics):
    fpga_energy_values = [metrics["fpga_energy"] for metrics in HW_metrics.values()]
    plt.hist(fpga_energy_values, bins=10)
    plt.xlabel("FPGA Energy")
    plt.ylabel("Frequency")
    plt.title("Histogram of FPGA Energy")
    plt.show()

                                                                        
                            

In [26]:
import json

def config():
    with open('config.json') as f:
        config_data = json.load(f)
    return config_data


In [19]:

import tqdm 
from hw_nas_bench_api import HWNASBenchAPI as HWAPI
hw_api = HWAPI("HW-NAS-Bench-v1_0.pickle", search_space="nasbench201")
data={}

# Example to get all the hardware metrics in the No.0,1,2 architectures under NAS-Bench-201's Space
print("===> Example to get all the hardware metrics in the No.0,1,2 architectures under NAS-Bench-201's Space")
engery_list=[]
latency_list=[]
network_info={}
for idx in tqdm.tqdm(range(15625)):
    for dataset in ["cifar10", "cifar100", "ImageNet16-120"]:
        HW_metrics = hw_api.query_by_index(idx, dataset)
        
        if HW_metrics["fpga_energy"] >0.02 and HW_metrics["fpga_latency"] < 0.5:
            network_info[idx]=dataset
#         engery_list.append(HW_metrics["fpga_energy"])
        latency_list.append(HW_metrics["fpga_latency"])
# data["engery"]=engery_list
# data["latency"]=latency_list
        # print("The HW_metrics (type: {}) for No.{} @ {} under NAS-Bench-201: {}".format(type(HW_metrics),
        #                                                                        idx,
        #                                                                        dataset,
        #                                                                        HW_metrics["fpga_energy"]))

===> Example to get all the hardware metrics in the No.0,1,2 architectures under NAS-Bench-201's Space


100%|██████████| 15625/15625 [00:00<00:00, 128286.89it/s]


In [20]:
print(f"mean latency: {sum(latency_list)/len(latency_list)} max latency: {max(latency_list)} min latency: {min(latency_list)}") 

mean latency: 2.8663451852799637 max latency: 8.67856 min latency: 0.28432


In [21]:
print(f"number of networks with energy >1 or latency >5: {len(network_info)}")

number of networks with energy >1 or latency >5: 688


In [27]:
from nas_201_api import NASBench201API as API
# Create an API without the verbose log
api = API('NAS-Bench-201-v1_0-e61699.pth', verbose=True)
# The default path for benchmark file is '{:}/{:}'.format(os.environ['TORCH_HOME'], 'NAS-Bench-201-v1_1-096897.pth')


try to create the NAS-Bench-201 api from NAS-Bench-201-v1_0-e61699.pth


UnpicklingError: pickle data was truncated

In [25]:
import torch
torch.load('NAS-Bench-201-v1_1-096897.pth',map_location=torch.device('cpu'))

UnpicklingError: pickle data was truncated

9.9456

In [2]:
# Example to get use the hardware metrics in the No.0 architectures in CIFAR-10 under NAS-Bench-201's Space
print("===> Example to get use the hardware metrics in the No.0 architectures in CIFAR-10 under NAS-Bench-201's Space")
HW_metrics = hw_api.query_by_index(0, "cifar10")
for k in HW_metrics:
    if 'average' in k:
        print("{}: {}".format(k, HW_metrics[k]))
        continue
    elif "latency" in k:
        unit = "ms"
    else:
        unit = "mJ"
    print("{}: {} ({})".format(k, HW_metrics[k], unit))
    

===> Example to get use the hardware metrics in the No.0 architectures in CIFAR-10 under NAS-Bench-201's Space
edgegpu_latency: 5.807418537139893 (ms)
edgegpu_energy: 24.226614330768584 (mJ)
raspi4_latency: 10.481976820010459 (ms)
edgetpu_latency: 0.9571811309997429 (ms)
pixel3_latency: 3.6058499999999998 (ms)
eyeriss_latency: 3.645620000000001 (ms)
eyeriss_energy: 0.6872827644999999 (mJ)
eyeriss_arithmetic_intensity: 6.289297634665844 (mJ)
fpga_latency: 2.57296 (ms)
fpga_energy: 18.01072 (mJ)
average_hw_metric: 591005.8676719831


In [13]:
# Create the network
config = hw_api.get_net_config(0, "cifar10")
print(config)
from hw_nas_bench_api.nas_201_models import get_cell_based_tiny_net
network = get_cell_based_tiny_net(config) # create the network from configurration
print(network) # show the structure of this architecture

{'name': 'infer.tiny', 'C': 16, 'N': 5, 'arch_str': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|', 'num_classes': 10}
TinyNetwork(
  TinyNetwork(C=16, N=5, L=17)
  (stem): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cells): ModuleList(
    (0-4): 5 x InferCell(
      info :: nodes=4, inC=16, outC=16, [1<-(I0-L0) | 2<-(I0-L1,I1-L2) | 3<-(I0-L3,I1-L4,I2-L5)], |avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|
      (layers): ModuleList(
        (0): POOLING(
          (op): AvgPool2d(kernel_size=3, stride=1, padding=1)
        )
        (1): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine

# Example to use HW-NAS-Bench under FBNet's Space

In [4]:
# The index in FBNet Space is not a number but a list with 22 elements, and each element is from 0~8
from hw_nas_bench_api import HWNASBenchAPI as HWAPI
hw_api = HWAPI("HW-NAS-Bench-v1_0.pickle", search_space="fbnet")

# Example to get all the hardware metrics in 3 specfic architectures under FBNet's Space
print("===> Example to get all the hardware metrics in the No.0,1,2 architectures under FBNet's Space")
for idx in [[0]*22, [0]*21+[1]*1, [0]*20+[1]*2]:
    for dataset in ["cifar100", "ImageNet"]:
        HW_metrics = hw_api.query_by_index(idx, dataset)
        print("The HW_metrics (type: {}) for No.{} @ {} under NAS-Bench-201: {}".format(type(HW_metrics),
                                                                               idx,
                                                                               dataset,
                                                                               HW_metrics))

===> Example to get all the hardware metrics in the No.0,1,2 architectures under FBNet's Space
The HW_metrics (type: <class 'dict'>) for No.[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] @ cifar100 under NAS-Bench-201: {'edgegpu_latency': 15.804523468017578, 'edgegpu_energy': 86.98095934380125, 'raspi4_latency': 17.618579145142576, 'pixel3_latency': 7.788973100000002, 'eyeriss_latency': 3.9408960000000004, 'eyeriss_energy': 3.2420080000000002, 'fpga_latency': 10.739520000000002, 'fpga_energy': 75.17664, 'average_hw_metric': 1945958271.4210253}
The HW_metrics (type: <class 'dict'>) for No.[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] @ ImageNet under NAS-Bench-201: {'edgegpu_latency': 16.431164979934692, 'edgegpu_energy': 102.71604275841212, 'raspi4_latency': 57.65650005702628, 'pixel3_latency': 27.5423871, 'eyeriss_latency': 18.127332000000003, 'eyeriss_energy': 16.945472000000002, 'fpga_latency': 29.42189999999999, 'fpga_energy': 205.9532999999

In [5]:
# Example to get use the hardware metrics in one specific architectures in ImageNet under FBNet's Space
print("===> Example to get use the hardware metrics in the No.0 architectures in ImageNet under FBNet's Space")
HW_metrics = hw_api.query_by_index([0]*22, "cifar100")
for k in HW_metrics:
    if 'average' in k:
        print("{}: {}".format(k, HW_metrics[k]))
        continue
    elif "latency" in k:
        unit = "ms"
    else:
        unit = "mJ"
    print("{}: {} ({})".format(k, HW_metrics[k], unit))
    

===> Example to get use the hardware metrics in the No.0 architectures in ImageNet under FBNet's Space
edgegpu_latency: 15.804523468017578 (ms)
edgegpu_energy: 86.98095934380125 (mJ)
raspi4_latency: 17.618579145142576 (ms)
pixel3_latency: 7.788973100000002 (ms)
eyeriss_latency: 3.9408960000000004 (ms)
eyeriss_energy: 3.2420080000000002 (mJ)
fpga_latency: 10.739520000000002 (ms)
fpga_energy: 75.17664 (mJ)
average_hw_metric: 1945958271.4210253


In [6]:
# Create the network
config = hw_api.get_net_config([0]*22, "cifar100")
print(config)
from hw_nas_bench_api.fbnet_models import FBNet_Infer
network = FBNet_Infer(config) # create the network from configurration
print(network) # show the structure of this architecture

{'dataset': 'cifar100', 'num_classes': 100, 'op_idx_list': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'arch_str': ['k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1', 'k3_e1']}
FBNet_Infer(
  (stem): ConvNorm(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (cells): ModuleList(
    (0): ConvBlock(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    